In [151]:
import streamlit as st
import pandas as pd
import numpy as np
import geopandas as gpd  # For handling GeoJSON data
import folium
import branca.colormap as cm
from streamlit_folium import folium_static  # Import to render folium maps in Streamlit
from sklearn.metrics import accuracy_score

In [161]:
fico_threshold = 650
energy_score_threshold = 0.5

solstice_territory_name = "Central Hudson"


def get_solstice_territory_geojson(solstice_territory_name):
    load_name = "filtered_geojsons/" + solstice_territory_name + '.geojson'
    temp = gpd.read_file(load_name)
    temp['Utility'] = solstice_territory_name
    return temp

zip_geojson = get_solstice_territory_geojson(solstice_territory_name)


person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(
    lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)


In [176]:
state_name

'Massachusetts'

In [175]:
def calculate_zip_metrics(stats_data_person, fico_threshold, energy_score_threshold):
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] > fico_threshold
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energy_score_threshold

    def calc_metrics(group):
        total_population = len(group)
        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })

        below_fico = group[group['FICO_PASS'] == False]
        above_fico = group[group['FICO_PASS'] == True]

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE']
                                        <= energy_score_threshold]
        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        percent_increase_in_qualifications = (
            len(below_fico_pass) / total_population) * 100 if len(below_fico_pass) > 0 else 0

        fico_accuracy = accuracy_score(
            below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['FICO_PASS']) if len(below_fico) > 0 else np.nan
        energy_accuracy = accuracy_score(
            below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['ENERGYSCORE_PASS']) if len(below_fico) > 0 else np.nan

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)
    zip_metrics = zip_metrics.reset_index()
    return zip_metrics


def calculate_zip_to_util(zip_level_geo, solstice_territory_name):

    # Load the utility data for the region selected
    state_util = get_solstice_territory_geojson(solstice_territory_name)


    zip_level_geo = pd.merge(zip_metrics, state_util, on='ZIP', how='left')
    zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
    return zip_level_geo




state_name = ""
calculate_zip_to_util()



In [165]:
state_util = get_solstice_territory_geojson(solstice_territory_name)

state_util.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,ZIP,geometry,Utility
0,12192,12192,B5,G6350,S,44437302.0,111097.0,+42.4090117,-073.8278389,12192,"MULTIPOLYGON (((-73.82379 42.36029, -73.82378 ...",Central Hudson
1,12193,12193,B5,G6350,S,100695328.0,910351.0,+42.5227595,-074.0432682,12193,"MULTIPOLYGON (((-74.11643 42.53985, -74.11592 ...",Central Hudson
2,12401,12401,B5,G6350,S,151378428.0,3445001.0,+41.9875407,-074.0102727,12401,"MULTIPOLYGON (((-74.1925 41.92533, -74.19213 4...",Central Hudson
3,12404,12404,B5,G6350,S,92428427.0,613979.0,+41.8189583,-074.2360822,12404,"MULTIPOLYGON (((-74.29932 41.84775, -74.28781 ...",Central Hudson
4,12405,12405,B5,G6350,S,18505054.0,75292.0,+42.3179089,-074.0859950,12405,"MULTIPOLYGON (((-74.15356 42.33362, -74.15338 ...",Central Hudson


In [172]:
zip_metrics = calculate_zip_metrics(
    person_data, fico_threshold, energy_score_threshold)

zip_metrics.head()

/var/folders/4_/36zlcy3x3m12nwb2nfhmkkjc0000gp/T/ipykernel_51815/1390087022.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)


,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,EnergyScore Accuracy,Qualification Increase
0,01001,11.0,0.181818,0.818182,1.0,1.0,18.181818
1,01002,12.0,0.166667,0.833333,1.0,1.0,16.666667
2,01005,3.0,0.000000,1.000000,NaN,NaN,0.000000
3,01007,18.0,0.055556,0.944444,1.0,1.0,5.555556
4,01010,3.0,0.333333,0.666667,1.0,1.0,33.333333


In [174]:
zip_level_geo = pd.merge(zip_metrics, state_util, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo.head()

# zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')

# zip_level_geo = zip_level_geo.to_crs(state_util.crs)

# zip_level_geo.head()

# zip_level_geo.Utility.value_counts()

print(zip_level_geo.shape)

zip_level_geo.head()

(92, 18)


,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,EnergyScore Accuracy,Qualification Increase,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,Utility
1979,10512,9.0,0.00,1.00,NaN,NaN,0.0,10512,10512,B5,G6350,S,141331607.0,14842472.0,+41.4576193,-073.7246077,"MULTIPOLYGON (((-73.82153 41.46554, -73.82151 ...",Central Hudson
1981,10516,2.0,0.00,1.00,NaN,NaN,0.0,10516,10516,B5,G6350,S,81626886.0,1587119.0,+41.4619726,-073.8749131,"MULTIPOLYGON (((-73.97859 41.44212, -73.9783 4...",Central Hudson
1987,10524,3.0,0.00,1.00,NaN,NaN,0.0,10524,10524,B5,G6350,S,53839856.0,839861.0,+41.3753251,-073.9262166,"MULTIPOLYGON (((-73.9795 41.32216, -73.97924 4...",Central Hudson
2014,10579,4.0,0.25,0.75,1.0,1.0,25.0,10579,10579,B5,G6350,S,84788614.0,3613306.0,+41.3953643,-073.8390532,"MULTIPOLYGON (((-73.89245 41.34375, -73.89193 ...",Central Hudson
2049,10916,4.0,0.25,0.75,1.0,1.0,25.0,10916,10916,B5,G6350,S,52319522.0,234779.0,+41.4423435,-074.2505767,"MULTIPOLYGON (((-74.32285 41.43885, -74.32243 ...",Central Hudson


In [160]:
zip_metrics = calculate_zip_metrics(
    person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')

zip_level_geo = zip_level_geo.to_crs(state_util.crs)

zip_level_geo['geometry'] = zip_level_geo.representative_point()

zip_level_geo = gpd.sjoin(
    zip_level_geo, state_util, how='left', predicate='within')


# Use the new function to calculate ZIP to utility metrics
state_util = calculate_zip_to_util(zip_level_geo, solstice_territory_name)

/var/folders/4_/36zlcy3x3m12nwb2nfhmkkjc0000gp/T/ipykernel_51815/1390087022.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)


ValueError: 'index_right' cannot be a column name in the frames being joined

In [ ]:
zip_level_geo.head()

In [ ]:
# test to combine the output files of Equifax and Experian person level data

eq_output = pd.read_csv('../energyscore-model/data/standard/equifax_rf_stats_person.csv')
eq_output['ZIP'] = eq_output['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')
eq_output.head()

print(eq_output.shape)

In [ ]:
ex_output = pd.read_csv('../energyscore-model/data/standard/experian_rf_stats_person.csv')
ex_output['ZIP'] = ex_output['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

ex_output.head()

print(eq_output.shape)


In [119]:
combined_columns = ['PROFILE_ID', 'NUMBER_OF_TRADELINES', 'WEIGHTED_ENERGYSCORE', 
                    'WEIGHTED_ACTUAL_OUTPUT', 'FICO_V9_SCORE', 'ZIP']


eq_output = eq_output[combined_columns]
ex_output = ex_output[combined_columns]

combined_df = pd.concat([ex_output, eq_output], ignore_index = True)
# combined_output = combined_output[combined_columns]
# combined_output.head()

In [ ]:
combined_df.shape

In [ ]:
combined_df.head()

In [2]:
# Load the GeoJSON file
zip_geojson = gpd.read_file('demo_zips.geojson')

# Load person data, forcing ZIP to be read as strings
person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)


In [20]:
energy_score_threshold=0.5

In [ ]:
person_data.shape

In [ ]:
person_data.head()

In [ ]:
person_data[person_data['ZIP']=='01001'].head(15)

In [ ]:
7/11

In [ ]:
zip_metrics.head()

In [60]:
# Function to calculate metrics for each ZIP
def calculate_zip_metrics(stats_data_person, fico_threshold, energy_score_threshold):
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] < fico_threshold
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energy_score_threshold

    def calc_metrics(group):
        total_population = len(group)

        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })
        
        above_fico = group[group['FICO_V9_SCORE'] > fico_threshold]
        below_fico = group[group['FICO_V9_SCORE'] < fico_threshold]

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energy_score_threshold]
        
        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100 if len(below_fico_pass) > 0 else 0
        numeric_increase_in_qualifications = len(below_fico_pass) if len(below_fico_pass) > 0 else 0

        energy_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['ENERGYSCORE_PASS']) #if len(below_fico) > 0 else np.nan

        fico_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['FICO_PASS']) #if len(below_fico) > 0 else np.nan

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
            'Numeric Increase': numeric_increase_in_qualifications,
        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)
    zip_metrics = zip_metrics.reset_index()
    return zip_metrics


# Function to calculate ZIP to utility mapping and display on the map
def calculate_zip_to_util(zip_level_geo, state_name):
    # Load the utility data for the state
    state_util = load_state_util(state_name)
    state_util.rename(columns={'new_name': 'Utility'}, inplace=True)

    # Ensure ZIP code geometries have the same projection as the utility data
    zip_level_geo = zip_level_geo.to_crs(state_util.crs)

    # Convert the ZIP geometries to representative points
    zip_level_geo['geometry'] = zip_level_geo.representative_point()

    # Perform spatial join with utility data based on point locations
    zip_level_geo = gpd.sjoin(zip_level_geo, state_util, how='left', predicate='within')

    # Group by utility name ('new_name') and calculate the mean of 'Qualification Increase'
    zip_to_util = zip_level_geo.groupby('Utility')['Qualification Increase'].mean().reset_index()

    # Merge utility data with the calculated qualification increase
    state_util = state_util.merge(zip_to_util, on='Utility', how='left')

    return state_util


In [67]:
state_name = "Massachusetts"
fico_threshold = 650
energy_score_threshold = .5


In [ ]:
zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')

print(zip_metrics['EnergyScore Accuracy'].describe())

print(zip_metrics['FICO Accuracy'].describe())


In [ ]:
zip_metrics.head()

In [ ]:
zip_metrics['EnergyScore Accuracy'].describe()

In [ ]:
zip_metrics['FICO Accuracy'].describe()

In [ ]:
zip_level_geo.head()

In [ ]:
import geopandas as gpd
import pandas as pd
import json
import os

# Step 1: Load the GeoJSON file containing all US ZIP codes
us_zip_geojson = gpd.read_file('/Users/jakeford/solstice/energyscore-model/us_zips.geojson')  # Adjust the file path as needed
us_zip_geojson['ZIP'] = us_zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)

# Step 2: Load the CSV file containing filter conditions for ZIP codes
df = pd.read_csv('//Users/jakeford/Downloads/82284_2024_10_16.csv')  # Load your CSV file

# Create a directory to save the output GeoJSON files
output_dir = 'filtered_geojsons'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [ ]:
df.head()

In [83]:
def extract_zipcodes(json_string):
    try:
        # Replace single quotes with double quotes to make it valid JSON
        json_string = json_string.replace("'", "\"")
        # Load the string as a dictionary and extract the list
        zipcodes_list = json.loads(json_string)['list']
        return zipcodes_list
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error decoding JSON: {e}")
        return []

# Apply the function to the 'zipcodes' column
df['zipcodes_list'] = df['zipcodes'].apply(extract_zipcodes)


In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    # Get the list of zip codes for the current row
    zipcodes_list = row['zipcodes_list']  # Assuming this column now contains the Python list of zip codes
    
    # Filter the GeoJSON data to include only the relevant ZIP codes
    filtered_zip_geojson = us_zip_geojson[us_zip_geojson['ZIP'].isin(zipcodes_list)]
    
    # Use the 'friendly_name' column for the filename
    friendly_name = row['friendly_name']
    output_filename = os.path.join(output_dir, f"{friendly_name}.geojson")
    
    # Save the filtered GeoJSON file
    filtered_zip_geojson.to_file(output_filename, driver='GeoJSON')
    
    print(f"Saved filtered GeoJSON for {friendly_name} to {output_filename}")

In [ ]:
fico_threshold = 650
energy_score_threshold = 0.5

In [ ]:
zip_geojson = gpd.read_file('demo_zips.geojson')

In [ ]:
df

In [ ]:
zip_level_geo.shape

In [ ]:
zip_level_geo['Total Population'].describe()

In [7]:
import fiona

In [ ]:
# Load the GeoJSON file
#zip_geojson = gpd.read_file('select_zips.geojson')

zip_geojson = gpd.read_file('demo_zips.geojson')

# Load person data, forcing ZIP to be read as strings
person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)

# Check if ZIPs were properly converted
print(person_data['ZIP'].head())
print(zip_geojson['ZIP'].head())


In [ ]:
# Function to calculate metrics for each ZIP
def calculate_zip_metrics(stats_data_person, fico_cutoff, energyscore_cutoff):
    # Create masks for conditions
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] > fico_cutoff
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff

    # Function to calculate various metrics
    def calc_metrics(group):
        total_population = len(group)
        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })
        
        
        below_fico = group[group['FICO_PASS'] == False]
        above_fico = group[group['FICO_PASS'] == True]

        if len(below_fico) == 0:
            return pd.Series({
                'Total Population': total_population,
                'Percent Below FICO': 0,
                'Percent Above FICO': len(above_fico) / total_population,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energyscore_cutoff]
        below_fico_fail = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff]

        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        if len(below_fico_pass) == 0:
            percent_increase_in_qualifications = 0
        else:
            percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100

        # get FICO accuracy, precision, recall, f1 and roc_auc score
       # fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff) if len(below_fico) > 0 else np.nan
        fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['FICO_PASS']) if len(below_fico) > 0 else np.nan

        energy_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['ENERGYSCORE_PASS']) if len(below_fico) > 0 else np.nan

      #  accuracy_increase = energy_accuracy - fico_accuracy

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
          #  'Accuracy Percentage Increase': accuracy_increase,

           # 'Accuracy Percentage Increase': (energy_accuracy - fico_accuracy) / fico_accuracy * 100 if fico_accuracy * energy_accuracy> 0 else 0,
           

        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)

    zip_metrics = zip_metrics.reset_index()
    return zip_metrics

fico_threshold = 700
energy_score_threshold = 0.5
# Calculate metrics and merge with geo data
zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')


In [ ]:
zip_level_geo

In [ ]:


def load_state_util(state_name):
    if state_name == 'New Mexico':
        temp = gpd.read_file('nm_utils.geojson')
        temp = temp[['new_name', 'geometry']]
        return temp
    elif state_name == 'Massachusetts':
        return gpd.read_file('ma_utils.geojson')


def calculate_zip_to_util(zip_level_geo, util_name, state_name):
    # Load the utility data for the state
    state_util = load_state_util(state_name)

    # Ensure ZIP code geometries have the same projection as the utility data
    zip_level_geo = zip_level_geo.to_crs(state_util.crs)

    # Convert the ZIP geometries to representative points (instead of centroids)
    zip_level_geo['geometry'] = zip_level_geo.representative_point()

    # Perform spatial join with utility data based on point locations using predicate
    zip_level_geo = gpd.sjoin(zip_level_geo, state_util, how='left', predicate='within')

    # Group by utility name ('new_name') and calculate the mean of 'Qualification Increase'
    zip_to_util = zip_level_geo.groupby('new_name')['Qualification Increase'].mean().reset_index()

    return zip_to_util

# Example usage
state_name = 'New Mexico'
util_name = 'Xcel'
zip_to_util = calculate_zip_to_util(zip_level_geo, util_name, state_name)
zip_to_util.head()


In [ ]:
zip_metrics

In [ ]:
zip_level_geo.shape

In [ ]:
zip_metrics['FICO Accuracy'].mean(),zip_metrics['EnergyScore Accuracy'].mean()

In [ ]:
zip_geojson.head()